# Final Submission

This file contains the code in our final submission. We use the same feature extraction techniques to generate our data, and then run LGBM in order to classify our data. 

In [1]:
# installing 1.0.3 because this version of pandas supports write to s3
!pip install pandas==1.0.3

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import re
import pickle
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
teamname = 'emotional-support-vector-machine-unsw'
data_folder='s3://tf-trachack-data/212/'
root_folder='s3://tf-trachack-notebooks/'+teamname+'/jupyter/jovyan/'

In [4]:
def cache(obj, filepath):
    """Cache the result"""
    directory = os.path.dirname(filepath)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(filepath, 'wb+') as f:
        pickle.dump(obj, f)

def is_cached(filepath):
    """Checks if the pickle file exists"""
    return os.path.isfile(filepath)


def load_cache(filepath):
    """Load the cached result"""
    assert is_cached(filepath)
    
    with open(filepath, 'rb') as f:
        return pickle.load(f)

## Feature Extraction

In [5]:
def minmax_scale(series):
    scaler = MinMaxScaler()
    return scaler.fit_transform(series.values.reshape(-1,1))

def standard_scale(series):
    scaler = StandardScaler()
    return scaler.fit_transform(series.values.reshape(-1,1))

def robust_scale(series):
    scaler = RobustScaler()
    return scaler.fit_transform(series.values.reshape(-1,1))

In [6]:
def convert_to_datetime(series):
    return pd.to_datetime(series, format="%Y-%m-%d")

In [7]:
lte_downlink = {0:1,1:10,2:50,3:100,4:150,5:300,6:300,7:300,8:3000,9:450,10:450,11:600,12:600,13:390,14:3900,15:800,16:1050,17:25000,18:1200,19:1600,20:2000,21:1400}
lte_uplink = {0:1,1:5,2:25,3:50,4:50,5:75,6:50,7:100,8:1500,9:50,10:100,11:50,12:100,13:150,14:1500,15:220,16:100,17:2100,18:210,19:13500,20:315,21:300}    

### 1. Upgrades

In [8]:
def extract1(upgrades, filepath=None, overwrite=False):
    upgrades = upgrades.drop('date_observed', axis=1)
    df1 = upgrades.replace({'no':0,'yes':1})
    return df1

### 2. Customer Info

In [9]:
def extract2(customer_info, filepath=None, overwrite=False):
    customer_info = customer_info.copy()
    
    df2 = customer_info.copy()
    df2 = pd.concat([df2, pd.get_dummies(df2['carrier'], prefix='carrier')], axis=1)

    date1 = pd.to_datetime(df2['first_activation_date'], format="%Y-%m-%d")
    date2 = pd.to_datetime(df2['redemption_date'], format="%Y-%m-%d")
    df2["days_to_redemption"] = np.log((date2 - date1).dt.days+1)
    df2['days_to_redemption'].fillna((df2['days_to_redemption'].median()), inplace=True)    
    scaler = MinMaxScaler()
    df2['days_to_redemption'] = scaler.fit_transform(df2['days_to_redemption'].values.reshape(-1,1))

    df2 = pd.concat([
        df2,
        pd.get_dummies(df2['plan_name'], prefix='plan')
    ], axis=1)

    df2.drop(
        ['carrier', 'first_activation_date', 'plan_subtype', 'redemption_date', 'plan_name'], 
        axis=1, inplace=True
    )

    return df2

### 3. Phone Info

In [10]:
def get_first_number(s):
    return int(re.search(r'\d+', str(s)).group())

In [11]:
def split_external_storage_capacity(s):
    s = str(s)
    if s == 'nan':
        return np.nan
    if '.' in s:
        return float(s)
    capacities = np.array(list(map(float, s.split('/'))))
    capacity = np.median(capacities)
    return capacity

In [12]:
def extract3(phone_info, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    phone_info = phone_info.copy()
    
    # get has_phone
    df3 = phone_info.copy()
    df3['has_phone'] = np.where(df3['cpu_cores'].isnull(), 0, 1)

    # get total cpus
    df3['cpu_cores'] = df3['cpu_cores'].dropna().apply(str).apply(eval)
    df3['cpu_cores'].fillna(df3['cpu_cores'].median(), inplace=True)
    scaler = MinMaxScaler()
    df3['cpu_cores'] = scaler.fit_transform(df3['cpu_cores'].values.reshape(-1,1))
    
    # expandable storage
    df3['expandable_storage'].fillna(0, inplace=True)
    
    # remove uncommon device types
    df3['gsma_device_type'].replace('WLAN Router', 'Other', inplace=True)
    df3['gsma_device_type'].replace('Tablet', 'Other', inplace=True)
    df3['gsma_device_type'].replace('Modem', 'Other', inplace=True)
    df3 = pd.concat([df3, pd.get_dummies(df3['gsma_device_type'], prefix='device_type')], axis=1)
    df3.drop(['gsma_device_type', 'device_type_Other'], axis=1, inplace=True)
    
    df3.drop(['gsma_model_name'], axis=1, inplace=True)
    
    # operating system
    df3['gsma_operating_system'].loc[
        (df3['gsma_operating_system'] != 'Android') &
        (df3['gsma_operating_system'] != 'iOS') &
        (df3['gsma_operating_system'] != 'Linux')
    ] = 'Other'
    df3 = pd.concat([df3, pd.get_dummies(df3['gsma_operating_system'], prefix='os')], axis=1)
    df3.drop(
        ['gsma_operating_system', 'os_family', 'os_name', 'os_vendor', 'os_version', 
         'manufacturer', 'os_Other'], axis=1, inplace=True)
    
    # internal stroage capacity, use medians
    df3['internal_storage_capacity'] = df3['internal_storage_capacity'].apply(split_external_storage_capacity)
    df3['internal_storage_capacity'].fillna(df3['internal_storage_capacity'].median(), inplace=True)
    df3['internal_storage_capacity'] = np.log2(df3['internal_storage_capacity'])
    scaler = StandardScaler()
    df3['internal_storage_capacity'] = scaler.fit_transform(df3['internal_storage_capacity'].values.reshape(-1,1))
    
    df3['lte'].fillna(0, inplace=True)
    df3['lte_advanced'].fillna(0, inplace=True)
    
    df3['lte_category'].fillna(df3['lte_category'].dropna().mode()[0], inplace=True)
    df3['lte_downlink'] = df3['lte_category'].map(lte_downlink)
    df3['lte_uplink'] = df3['lte_category'].map(lte_uplink)
    scaler = RobustScaler()
    df3['lte_downlink'] = scaler.fit_transform(df3['lte_downlink'].values.reshape(-1,1))
    df3['lte_uplink'] = scaler.fit_transform(df3['lte_uplink'].values.reshape(-1,1))
    df3['lte_downlink'] /= 100
    df3['lte_uplink'] /= 100
    df3.drop(['lte_category'], axis=1, inplace=True)
    
    df3.drop(['sim_size'], axis=1, inplace=True)
    
    df3['total_ram'] = df3['total_ram'].apply(split_external_storage_capacity)
    df3['total_ram'].fillna(df3['total_ram'].median(), inplace=True)
    df3['total_ram'] = np.log2(df3['total_ram'])
    scaler = MinMaxScaler()
    df3['total_ram'] = scaler.fit_transform(df3['total_ram'].values.reshape(-1,1))
    df3['touch_screen'].fillna(0, inplace=True)
    df3['wi_fi'].fillna(0, inplace=True)
        
    # years since release
    df3 = pd.merge(df3, upgrades[['line_id', 'date_observed']].copy(), on='line_id')
    df3['date_observed'] = pd.to_datetime(df3['date_observed'], format="%Y-%m-%d").dt.year
    
    df3['years_since_release'] = df3['date_observed'] - df3['year_released']
    df3['years_since_release'].fillna(df3['years_since_release'].dropna().median(), inplace=True)
    df3['years_since_release'] = minmax_scale(df3['years_since_release'])
    
    df3.drop(['date_observed', 'year_released'], axis=1, inplace=True)
    
    cache(df3, filepath)
    
    return df3

### 4. Redemptions

In [13]:
def extract4(redemptions, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    redemptions = redemptions.copy()
    df4 = upgrades[['line_id']].copy()
    
    #num redemptions
    num_redemptions = redemptions.groupby(['line_id']).size().to_frame('num_redemptions')
    df4 = pd.merge(df4, num_redemptions, on='line_id', how='left')
    df4['num_redemptions'].fillna(0, inplace=True)
    df4['num_redemptions'] = np.log(df4['num_redemptions']+1)
    scaler = MinMaxScaler()
    df4['num_redemptions'] = scaler.fit_transform(df4['num_redemptions'].values.reshape(-1,1))
    
    # num one offs
    one_off_threshold = 15
    num_one_offs = redemptions[redemptions['gross_revenue'] <= one_off_threshold].groupby(['line_id']).size().to_frame('num_one_offs')
    df4 = pd.merge(df4, num_one_offs, on='line_id', how='left')
    df4['num_one_offs'].fillna(0, inplace=True)
    df4['num_one_offs'] = np.log(df4['num_one_offs']+1)
    scaler = MinMaxScaler()
    df4['num_one_offs'] = scaler.fit_transform(df4['num_one_offs'].values.reshape(-1,1))
    
    # total spent
    total_spent = redemptions.groupby(['line_id'])['gross_revenue'].sum().to_frame('total_spent')
    df4 = pd.merge(df4, total_spent, on='line_id', how='left')
    df4['total_spent'].fillna(0, inplace=True)
    
    # average monthly
    redemptions['redemption_date'] = pd.to_datetime(redemptions['redemption_date'], format="%Y-%m-%d")
    date_start = redemptions.groupby(['line_id'])['redemption_date'].min().to_frame('date_start')
    df4 = pd.merge(df4, date_start, on='line_id', how='left')
    date_end = redemptions.groupby(['line_id'])['redemption_date'].max().to_frame('date_end')
    df4 = pd.merge(df4, date_end, on='line_id', how='left')
    df4['date_start'].fillna(np.datetime64('1970'), inplace=True)
    df4['date_end'].fillna(np.datetime64('1970'), inplace=True)
    
    df4['total_months'] = (df4['date_end'] - df4['date_start']).dt.days//30+1
    df4['average_monthly_spend'] = df4['total_spent'] / df4['total_months']
    df4.drop(['date_start', 'date_end', 'total_months'], axis=1, inplace=True)
    
    df4['total_spent'] = np.log(df4['total_spent']+1)
    scaler = StandardScaler()
    df4['total_spent'] = scaler.fit_transform(df4['total_spent'].values.reshape(-1,1))
    
    df4['average_monthly_spend'] = np.log(df4['average_monthly_spend']+1)
    scaler = MinMaxScaler()
    df4['average_monthly_spend'] = scaler.fit_transform(df4['average_monthly_spend'].values.reshape(-1,1))
    
    # monthly plan cost
    monthly_plan_cost = redemptions[redemptions['gross_revenue'] > one_off_threshold].groupby(['line_id']).median()
    monthly_plan_cost = monthly_plan_cost.rename(columns={'gross_revenue':'monthly_plan_cost'})
        
    df4 = pd.merge(df4, monthly_plan_cost, on='line_id', how='left')
    df4['monthly_plan_cost'].fillna(0, inplace=True)
    df4['monthly_plan_cost'] = np.log(df4['num_one_offs']+1)
    scaler = MinMaxScaler()
    df4['monthly_plan_cost'] = scaler.fit_transform(df4['num_one_offs'].values.reshape(-1,1))
    
    cache(df4, filepath)
    
    return df4

### 5. Deactivations

In [14]:
def extract5(deactivations, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    deactivations = deactivations.copy()
    df5 = upgrades[['line_id']].copy()
    
    # has deactivated
    has_deactivated = deactivations.groupby('line_id')['deactivation_reason'].any()
    has_deactivated = has_deactivated.rename(f'has_deactivated')
    df5 = pd.merge(df5, has_deactivated, on='line_id', how='left')
    df5[f'has_deactivated'].replace({np.nan:0, True:1}, inplace=True)
    
    # has certain deactivation reasons
    useful_values = ["PASTDUE", "UPGRADE", "DEVICE CHANGE INQUIRY", "STOLEN", 
                    "DEFECTIVE", "ACTIVE UPGRADE", "REFURBISHED", "NO NEED OF PHONE", "DEVICERETURN"]
    for useful_value in useful_values: 
        has_value = deactivations[deactivations['deactivation_reason'] == useful_value].groupby('line_id')['deactivation_reason'].any()
        has_value = has_value.rename(f'reason_{useful_value}')
        df5 = pd.merge(df5, has_value, on='line_id', how='left')
        df5[f'reason_{useful_value}'].replace({np.nan:0, True:1}, inplace=True)

    # num deactivations
    num_deactivations = deactivations.groupby(['line_id']).size().to_frame('num_deactivations')
    df5 = pd.merge(df5, num_deactivations, on='line_id', how='left')
    df5['num_deactivations'].fillna(0, inplace=True)
    df5['num_deactivations'] = np.log(df5['num_deactivations']+1)
    df5['num_deactivations'] = minmax_scale(df5['num_deactivations'])
        
    cache(df5, filepath)
        
    return df5

### 6. Reactivations

In [15]:
def extract6(reactivations, deactivations, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    reactivations = reactivations.copy()
    df6 = upgrades[['line_id']].copy()
    
    # check if active
    df6['is_active'] = 1
    
    deactivations['deactivation_date'] = pd.to_datetime(deactivations['deactivation_date'], format="%Y-%m-%d")
    reactivations['reactivation_date'] = pd.to_datetime(reactivations['reactivation_date'], format="%Y-%m-%d")
    
    last_deactivate = deactivations.groupby(['line_id'])['deactivation_date'].max().to_frame('last_deactivate')
    last_reactivate = reactivations.groupby(['line_id'])['reactivation_date'].max().to_frame('last_reactivate')
    
    df6 = pd.merge(df6, last_deactivate, on='line_id', how='left')
    df6 = pd.merge(df6, last_reactivate, on='line_id', how='left')
    
    has_deactivated = deactivations.groupby('line_id')['deactivation_reason'].any()
    has_deactivated = has_deactivated.rename(f'has_deactivated')
    df6 = pd.merge(df6, has_deactivated, on='line_id', how='left')
    df6['has_deactivated'].replace({np.nan:0, True:1}, inplace=True)
    df6['reactivated'] = np.where(df6['last_reactivate'] - df6['last_deactivate'] >= np.timedelta64(0, 'D'), 1, 0)
    
    df6['is_active'] = 1 - df6[f'has_deactivated'] + df6['reactivated']
    
    df6.drop(['last_deactivate', 'has_deactivated', 'reactivated'], axis=1, inplace=True)
    
    #num reactivations
    num_reactivations = reactivations.groupby(['line_id']).size().to_frame('num_reactivations')
    df6 = pd.merge(df6, num_reactivations, on='line_id', how='left')
    df6['num_reactivations'].fillna(0, inplace=True)
    df6['num_reactivations'] = np.log(df6['num_reactivations']+1)
    df6['num_reactivations'] = minmax_scale(df6['num_reactivations'])
    
    # days since last reactivation
    df6 = pd.merge(df6, upgrades[['line_id','date_observed']], on='line_id', how='left')
    
    df6['date_observed'] = convert_to_datetime(df6['date_observed'])
    df6['days_since_reactivation'] = (df6['date_observed'] - df6['last_reactivate']).dt.days
    df6['days_since_reactivation'] = minmax_scale(np.log(df6['days_since_reactivation']+1))
    df6['days_since_reactivation'].fillna(0, inplace=True)
    
    df6.drop(['date_observed', 'last_reactivate'], axis=1, inplace=True)
        
    cache(df6, filepath)
    
    return df6

### 7. Suspensions

In [16]:
def extract7(suspensions, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    suspensions = suspensions.copy()
    df7 = upgrades[['line_id', 'date_observed']].copy()
    
    # num and has been suspended
    num_suspensions = suspensions.groupby(['line_id']).size().to_frame('num_suspensions')
    df7 = pd.merge(df7, num_suspensions, on='line_id', how='left');
    df7['num_suspensions'].fillna(0, inplace=True)
    df7['has_been_suspended'] = np.where(df7['num_suspensions'] > 0, 1, 0)
    df7['num_suspensions'] = minmax_scale(np.log(df7['num_suspensions']+1))
    
    # months since last suspension

    df7['date_observed'] = convert_to_datetime(df7['date_observed'])
    suspensions['suspension_start_date'] = convert_to_datetime(suspensions['suspension_start_date'])
    suspensions['suspension_end_date'] = convert_to_datetime(suspensions['suspension_end_date'])

    last_suspension_date = suspensions.groupby('line_id')['suspension_end_date'].max()
    last_suspension_date.rename('last_suspension_date')
    df7 = pd.merge(df7, last_suspension_date, on='line_id', how='left')
    
    df7['months_since_suspended'] = (df7['date_observed'] - df7['suspension_end_date']).dt.days/30+1
    df7['months_since_suspended'].fillna(0, inplace=True)
    df7['months_since_suspended'] = minmax_scale(np.log(df7['months_since_suspended']+1))
    
    df7.drop(['date_observed', 'suspension_end_date'], axis=1, inplace=True)
    
    # average suspension length
    suspensions['suspension_length'] = (suspensions['suspension_end_date'] - suspensions['suspension_start_date']).dt.days
    average_suspension_length = suspensions.groupby('line_id')['suspension_length'].mean().to_frame('average_suspension_length')
    df7 = pd.merge(df7, average_suspension_length, on='line_id', how='left')
    df7['average_suspension_length'].fillna(0, inplace=True)
    df7['average_suspension_length'] = minmax_scale(df7['average_suspension_length']+1)
    
    cache(df7, filepath)
    return df7

### 8. Network Usage

In [17]:
def extract8(network_usage, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    network_usage = network_usage.copy()
    df8 = upgrades[['line_id']].copy()
    
    # sms/mms in and out
    print("  sms and mms...")
    sms_in_total = (network_usage.groupby('line_id')['mms_in'].sum() +
         network_usage.groupby('line_id')['sms_in'].sum()).to_frame('sms_in_total')
    df8 = pd.merge(df8, sms_in_total, on='line_id', how='left')
    df8['sms_in_total'].fillna(0, inplace=True)    
    
    sms_out_total = (network_usage.groupby('line_id')['mms_out'].sum() +
         network_usage.groupby('line_id')['sms_out'].sum()).to_frame('sms_out_total')
    df8 = pd.merge(df8, sms_out_total, on='line_id', how='left')
    df8['sms_out_total'].fillna(0, inplace=True)  
    
#     df8['sms_diff'] = df8['sms_in_total'] - df8['sms_out_total']
    df8['sms_total'] = df8['sms_in_total'] + df8['sms_out_total']
    
    df8['over_15_sms'] = np.choose(df8['sms_total'] >= 15, [0, 1])
    
    # kilobytes of data
    print("  kilobytes of data...")
    hotspot_kb = network_usage.groupby('line_id')['hotspot_kb'].sum()
    df8 = pd.merge(df8, hotspot_kb, on='line_id', how='left')
    df8['hotspot_kb'].fillna(0, inplace=True)  
    df8['hotspot_gb'] = df8['hotspot_kb'] / 1024.0 / 1024.0
    
    kb_5g = network_usage.groupby('line_id')['kb_5g'].sum()
    df8 = pd.merge(df8, kb_5g, on='line_id', how='left')
    df8['kb_5g'].fillna(0, inplace=True) 
    df8['gb_5g'] = df8['kb_5g'] / 1024.0 / 1024.0
             
    total_kb = network_usage.groupby('line_id')['total_kb'].sum()
    df8 = pd.merge(df8, total_kb, on='line_id', how='left')
    df8['total_kb'].fillna(0, inplace=True)  
    df8['total_gb'] = df8['total_kb'] / 1024.0 / 1024.0
    
    df8.drop(['hotspot_kb', 'kb_5g', 'total_kb'], axis=1, inplace=True)
    
    # voice counts
    print("  voice counts...")
    voice_count_in = network_usage.groupby('line_id')['voice_count_in'].sum()
    df8 = pd.merge(df8, voice_count_in, on='line_id', how='left')
    df8['voice_count_in'].fillna(0, inplace=True) 
    
    voice_count_total = network_usage.groupby('line_id')['voice_count_total'].sum()
    df8 = pd.merge(df8, voice_count_total, on='line_id', how='left')
    df8['voice_count_total'].fillna(0, inplace=True) 
    
    df8['voice_count_out'] = df8['voice_count_total'] - df8['voice_count_in']
#     df8['voice_count_diff'] = df8['voice_count_in'] - df8['voice_count_out']  
        
    df8['over_5_voice'] = np.choose(df8['voice_count_total'] >= 5, [0, 1])
        
    # voice minutes
    print("  voice minutes..")
    voice_min_in = network_usage.groupby('line_id')['voice_min_in'].sum()
    df8 = pd.merge(df8, voice_min_in, on='line_id', how='left')
    df8['voice_min_in'].fillna(0, inplace=True) 
    
    voice_min_out = network_usage.groupby('line_id')['voice_min_out'].sum()
    df8 = pd.merge(df8, voice_min_out, on='line_id', how='left')
    df8['voice_min_out'].fillna(0, inplace=True) 
    
    df8['voice_min_total'] = df8['voice_min_in'] + df8['voice_min_out']
    
    # average call length
    df8['average_call_length'] = np.choose(
        df8['voice_count_total'] == 0, 
        [df8['voice_min_total'] / df8['voice_count_total'], 0]
    )
    
    #num usages
    num_network_usages = network_usage.groupby(['line_id']).size().to_frame('num_network_usages')
    df8 = pd.merge(df8, num_network_usages, on='line_id', how='left')
    df8['num_network_usages'].fillna(0, inplace=True)
    df8['num_network_usages'] = np.log(df8['num_network_usages']+1)
    df8['num_network_usages'] = minmax_scale(df8['num_network_usages'])
    
    print("  scaling all parameters..")
    df8['sms_in_total'] = standard_scale(np.log(df8['sms_in_total']+1))
    df8['sms_out_total'] = standard_scale(np.log(df8['sms_out_total']+1))
    df8['sms_total'] = standard_scale(np.log(df8['sms_total']+1))
#     df8['sms_diff'] = standard_scale(df8['sms_diff'])
    df8['hotspot_gb'] = standard_scale(np.log(df8['hotspot_gb']+1))
    df8['gb_5g'] = standard_scale(np.log(df8['gb_5g']+1))
    df8['total_gb'] = standard_scale(np.log(df8['total_gb']+1))
    df8['voice_count_in'] = standard_scale(np.log(df8['voice_count_in']+1))
    df8['voice_count_out'] = standard_scale(np.log(df8['voice_count_out']+1))
    df8['voice_count_total'] = standard_scale(np.log(df8['voice_count_total']+1))
    df8['voice_min_in'] = standard_scale(np.log(df8['voice_min_in']+1))
    df8['voice_min_out'] = standard_scale(np.log(df8['voice_min_out']+1))
    df8['voice_min_total'] = standard_scale(np.log(df8['voice_min_total']+1))
#     df8['voice_count_diff'] = standard_scale(df8['voice_count_diff'])
    df8['average_call_length'] = standard_scale(np.log(df8['average_call_length']+1))
    
    cache(df8, filepath)
    
    return df8

### 9. LRP Points

In [18]:
def extract9(lrp_points, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    lrp_points = lrp_points.copy()
    df9 = upgrades[['line_id']].copy()
    
    df9 = pd.merge(df9, lrp_points, on='line_id', how='left')
    
    # total_quantity
    df9['total_quantity'].fillna(0, inplace=True)

    # has_earned_points
    df9['has_earned_points'] = np.where(df9['status'] == 'ENROLLED', 1, 0)
    
    # total_spent
    df9['total_spent_lrp'] = df9['total_quantity'] - df9['quantity']
    df9['total_spent_lrp'].fillna(0, inplace=True)
    
    # has_spent
    df9['has_spent'] = np.where(df9['total_spent_lrp'] > 0, 1, 0)
    
    # days_since_earnt
    df9['update_date'] = convert_to_datetime(df9['update_date'])
    df9 = pd.merge(df9, upgrades[['line_id', 'date_observed']].copy(), on='line_id', how='left')
    df9['date_observed'] = convert_to_datetime(df9['date_observed'])
    
    df9['days_since_update'] = (df9['date_observed'] - df9['update_date']).dt.days
    df9['days_since_update'].fillna(0, inplace=True)
    df9['days_since_update'] = minmax_scale(np.log(df9['days_since_update']+1))
    
    df9.drop(['quantity', 'status', 'update_date', 'date_observed'], axis=1, inplace=True)

    cache(df9, filepath)
    return df9

### 10. LRP Enrollment

In [19]:
def extract10(lrp_enrollment, upgrades, filepath=None, overwrite=False):
    if is_cached(filepath) and not overwrite:
        return load_cache(filepath)
    
    lrp_enrollment = lrp_enrollment.copy()
    df10 = upgrades[['line_id', 'date_observed']].copy()
    
    num_enrols = lrp_enrollment.groupby('line_id').size().to_frame('num_enrols')
    df10 = pd.merge(df10, num_enrols, on='line_id', how='left')
    df10['num_enrols'].fillna(0, inplace=True)
    df10['has_enrolled'] = np.where(df10['num_enrols'] > 0, 1, 0)
    df10['has_enrolled_over_twice'] = np.where(df10['num_enrols'] >= 2, 1, 0)
    
    df10['num_enrols'] = minmax_scale(df10['num_enrols'])
    
    # num months since enrolment
    df10['date_observed'] = convert_to_datetime(df10['date_observed'])
    lrp_enrollment['lrp_enrollment_date'] = convert_to_datetime(lrp_enrollment['lrp_enrollment_date'])
    
    last_enrolled_date = lrp_enrollment.groupby('line_id')['lrp_enrollment_date'].max()
    df10 = pd.merge(df10, last_enrolled_date, on='line_id', how='left')
    
    df10['months_since_enrolled'] = (df10['date_observed'] - df10['lrp_enrollment_date']).dt.days/30+1
    df10['months_since_enrolled'].fillna(0, inplace=True)
    df10['months_since_enrolled'] = standard_scale(np.log(df10['months_since_enrolled']+1))
    
    df10.drop(['date_observed', 'lrp_enrollment_date'], axis=1, inplace=True)
    
    cache(df10, filepath)
    return df10

### Merging

In [20]:
def load_data(data="dev"):
    assert(data == "dev" or data == "eval")
    
    if data == "dev":
        base_folder = data_folder+"data/dev/"
    elif data == "eval":
        base_folder = data_folder+"data/eval/"
        
    print("loading upgrades...")
    upgrades = pd.read_csv(base_folder + "upgrades.csv")
    print("loading customer_info...")
    customer_info = pd.read_csv(base_folder + "customer_info.csv")
    print("loading phone_info...")
    phone_info = pd.read_csv(base_folder + "phone_info.csv")
    print("loading redemptions...")
    redemptions = pd.read_csv(base_folder + "redemptions.csv")
    print("loading deactivations...")
    deactivations = pd.read_csv(base_folder + "deactivations.csv")
    print("loading reactivations...")
    reactivations = pd.read_csv(base_folder + "reactivations.csv")
    print("loading suspensions...")
    suspensions = pd.read_csv(base_folder + "suspensions.csv")
    print("loading network_usage_domestic...")
    network_usage_domestic = pd.read_csv(base_folder + "network_usage_domestic.csv")
    print("loading lrp_points...")
    lrp_points = pd.read_csv(base_folder + "lrp_points.csv")
    print("loading lrp_enrollment...")
    lrp_enrollment = pd.read_csv(base_folder + "lrp_enrollment.csv")
    
    print("done!")
    
    return upgrades, customer_info, phone_info, redemptions, deactivations, reactivations, \
        suspensions, network_usage_domestic, lrp_points, lrp_enrollment

In [21]:
data_dev = load_data('dev')
data_eval = load_data('eval')

print("length of dev:", len(data_dev))
print("length of eval:", len(data_eval))

loading upgrades...
loading customer_info...
loading phone_info...
loading redemptions...
loading deactivations...
loading reactivations...
loading suspensions...
loading network_usage_domestic...
loading lrp_points...
loading lrp_enrollment...
done!
loading upgrades...
loading customer_info...
loading phone_info...
loading redemptions...
loading deactivations...
loading reactivations...
loading suspensions...
loading network_usage_domestic...
loading lrp_points...
loading lrp_enrollment...
done!
length of dev: 10
length of eval: 10


In [22]:
def extract_features(datasets, data="dev", overwrite=False):
    assert(data == "dev" or data == "eval")
    
    if data == "dev":
        base_path = root_folder+"code/pickles/{}_dev.pickle"
        output_path = root_folder+"code/dev-extracted.csv"
        
    elif data == "eval":
        base_path = root_folder+"code/pickles/{}_eval.pickle"
        output_path = root_folder+"code/eval-extracted.csv"
        
    (upgrades, customer_info, phone_info, redemptions, deactivations, reactivations, \
        suspensions, network_usage_domestic, lrp_points, lrp_enrollment) = datasets
    
    print("extracting 1...")
    df1 = extract1(upgrades)
    print("extracting 2...")
    df2 = extract2(customer_info, upgrades)
    print("extracting 3...")
    df3 = extract3(phone_info, upgrades, filepath=base_path.format(3), overwrite=overwrite)
    print("extracting 4...")
    df4 = extract4(redemptions, upgrades, filepath=base_path.format(4), overwrite=overwrite)
    print("extracting 5...")
    df5 = extract5(deactivations, upgrades, filepath=base_path.format(5), overwrite=overwrite)
    print("extracting 6...")
    df6 = extract6(reactivations, deactivations, upgrades, filepath=base_path.format(6), overwrite=overwrite)
    print("extracting 7...")
    df7 = extract7(suspensions, upgrades, filepath=base_path.format(7), overwrite=overwrite)
    print("extracting 8...")
    df8 = extract8(network_usage_domestic, upgrades, filepath=base_path.format(8), overwrite=overwrite)
    print("extracting 9...")
    df9 = extract9(lrp_points, upgrades, filepath=base_path.format(9), overwrite=overwrite)
    print("extracting 10...")
    df10 = extract10(lrp_enrollment, upgrades, filepath=base_path.format(10), overwrite=overwrite)
    
    print("extracted all features")
    
    print("merging them all together...")
    dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
    df = dfs[0].copy()
    for i in range(1, len(dfs)):
        assert(len(dfs[i]) == len(df))
        df = pd.merge(df, dfs[i], on='line_id')
    
    assert(df.isna().any().any() == False)
        
    print("saving to file...")
    print(output_path)
    df.to_csv(output_path, header=True, index=None)
    
    print("done!")
    
    return df

In [23]:
data_dev_extracted = extract_features(data_dev, 'dev', overwrite=True)
data_eval_extracted = extract_features(data_eval, 'eval', overwrite=True)

print("number of features:", len(data_dev_extracted.dtypes))

extracting 1...
extracting 2...
extracting 3...
extracting 4...
extracting 5...
extracting 6...
extracting 7...
extracting 8...
  sms and mms...
  kilobytes of data...
  voice counts...
  voice minutes..
  scaling all parameters..
extracting 9...
extracting 10...
extracted all features
merging them all together...
saving to file...
s3://tf-trachack-notebooks/emotional-support-vector-machine-unsw/jupyter/jovyan/code/dev-extracted.csv
done!
extracting 1...
extracting 2...
extracting 3...
extracting 4...
extracting 5...
extracting 6...
extracting 7...
extracting 8...
  sms and mms...
  kilobytes of data...
  voice counts...
  voice minutes..
  scaling all parameters..
extracting 9...
extracting 10...
extracted all features
merging them all together...
saving to file...
s3://tf-trachack-notebooks/emotional-support-vector-machine-unsw/jupyter/jovyan/code/eval-extracted.csv
done!
number of features: 77


## Classification

In [24]:
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [25]:
y = data_dev_extracted['upgrade']
X = data_dev_extracted.drop(['line_id', 'upgrade'], axis = 1)

print("num features:", len(X.dtypes))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
print("num train samples:", len(X_train))
print("num test samples:", len(X_test))

num features: 75
num train samples: 44694
num test samples: 11174


In [26]:
lg = lgb.LGBMClassifier(learning_rate=0.1,
                        n_estimators=400,
                        max_bin=255,
                        max_depth=40, 
                        num_iterations=400, 
                        random_state=11,
                        num_leaves=150,
                        silent=False)

print("training LGBM...")
t0 = time.time()
lg.fit(X, y, eval_set=(X_test, y_test), verbose=True) 
# logloss printed is meaningless, since X includes X_test, just here to print something and show progress
print(f"time to train: {time.time()-t0:.4}s")

# yhat = lg.predict(X_test)
# print("f1_score:", f1_score(y_test, yhat))

training LGBM...


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 15174, number of negative: 40694
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011050 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5931
[LightGBM] [Info] Number of data points in the train set: 55868, number of used features: 69
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271604 -> initscore=-0.986497
[LightGBM] [Info] Start training from score -0.986497
[1]	valid_0's binary_logloss: 0.514494
[2]	valid_0's binary_logloss: 0.463968
[3]	valid_0's binary_logloss: 0.423555
[4]	valid_0's binary_logloss: 0.389695
[5]	valid_0's binary_logloss: 0.361414
[6]	valid_0's binary_logloss: 0.337302
[7]	valid_0's binary_logloss: 0.316563
[8]	va

In [27]:
upgrade_eval = pd.read_csv(data_folder+"data/eval/upgrades.csv")
line_ids = upgrade_eval[['line_id']]
prediction = line_ids.copy()

X_eval = data_eval_extracted.drop(['line_id'], axis=1)
prediction['upgrade'] = lg.predict(X_eval)

assert(len(prediction) == len(upgrade_eval))
print("length of prediction:", len(prediction))

submission_path=root_folder+"code/2021-04-25.csv"
prediction.to_csv(submission_path, header=True, index=None)

submission_path=root_folder+"submission/2021-04-25.csv"
prediction.to_csv(submission_path, header=True, index=None)

print("submitted!")

length of prediction: 37155
submitted!
